# Makes some test on the Laplacian, Biharmonic and Heisenberg layers

The second derivatives of the chi matrix are used to evaluate the expected number of photons in each channels
The fourth derivatives are used to determin the uncertainties in the photon number

Here we test the uncertainty layer on different states

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>

nonlinearxwaves@gmail.com<br>
@created 24 december 2020<br>
@version 23 sep 2023

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages

In [2]:
import numpy as np
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
from tensorflow import keras

In [3]:
tf.keras.backend.clear_session()

In [4]:
np.set_printoptions(precision=2)

In [5]:
tf_real=tf.float32

In [6]:
tf.keras.backend.set_floatx('float32')

## Dimension (number of modes times 2, N=2n)

In [7]:
N = 4

## Build vacuum by the Gaussian state

In [8]:
vacuum = ps.VacuumLayer(N)

We use a vacuum layer with modified covariance matrix g to test derivatives

In [9]:
g=np.eye(N); 
for i in range(N):
    g[i,i]=i+1
print(g)

[[1. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 4.]]


In [10]:
ModifiedVacuum=ps.GaussianLayer(g, np.zeros((N,1)), trainable=False, dtype=tf_real)

In [11]:
ModifiedVacuum.g

<tf.Variable 'gaussian_layer_1/g:0' shape=(4, 4) dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 2., 0., 0.],
       [0., 0., 3., 0.],
       [0., 0., 0., 4.]], dtype=float32)>

## Squeezed mode on mode 0

In [12]:
ra=0.1;
phia=np.pi/2;

In [13]:
squeezer=ps.SingleModeSqueezerLayer(N, r_np=ra, theta_np=phia, n_squeezed=0, trainable=False)

## Coherent state (Displacer on mode 1)

In [14]:
A_np = 3.14
lambda_np = np.pi/2
#theta =0
dinput=np.zeros((N,1));
#dinput[2]=np.sqrt(2)*A_np*np.cos(lambda_np);
#dinput[3]=np.sqrt(2)*A_np*np.sin(lambda_np);
displacer=ps.DisplacementLayerConstant(dinput)

### Dummy training points (not used for training in this example)

In [15]:
Nbatch=10
gtarget=np.eye(N)
dtarget=2.4*np.ones((N,1))
xtrain = np.random.rand(Nbatch, N)-0.5
ytrain =np.zeros_like(xtrain)
dtrain = np.zeros((Nbatch,N))
gtrain = np.zeros((Nbatch,N,N))
for j in range(Nbatch):
    for i in range(N):
        dtrain[j,i]=dtarget[i]
        for k in range(N):
            gtrain[j,i,k]=gtarget[i,k]

# Model for the vacuum state

In [16]:
xin = tf.keras.layers.Input(N)
chir, chii = ModifiedVacuum(xin)
modelVacuum = tf.keras.Model(inputs = xin, outputs=[chir, chii])

# Details the operation to evaluate the Laplacian

## Evaluate the Laplacian 

In [17]:
x = tf.Variable(np.zeros((1,N)), dtype=tf_real)
with tf.GradientTape() as t1:
    with tf.GradientTape() as t2:
        cr, _ = modelVacuum(x)
    cr_x = t2.gradient(cr,x)
cr_xy = t1.jacobian(cr_x,x)

Here cr has shape [1,1], cr_x has shape [1,N], and cr_xy has shape [1,N,1,N,1,N]

In [18]:
print(cr.shape); print(cr_x.shape);print(cr_xy.shape)

(1, 1)
(1, 4)
(1, 4, 1, 4)


In [19]:
print(cr); print(cr_x);print(cr_xy)

tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
tf.Tensor([[0. 0. 0. 0.]], shape=(1, 4), dtype=float32)
tf.Tensor(
[[[[-0.5  0.   0.   0. ]]

  [[ 0.  -1.   0.   0. ]]

  [[ 0.   0.  -1.5  0. ]]

  [[ 0.   0.   0.  -2. ]]]], shape=(1, 4, 1, 4), dtype=float32)


Build a list of indices to extract the diagonal part of the Hessian 

In [20]:
list1_np=[]
for i in range(N):
    list1_np.append([0,i]*2)
print(list1_np)
list1=tf.constant(list1_np)
list1.shape

[[0, 0, 0, 0], [0, 1, 0, 1], [0, 2, 0, 2], [0, 3, 0, 3]]


TensorShape([4, 4])

Gather the diagonal elements by the list of indices

In [21]:
cr_xx=tf.reshape(tf.gather_nd(cr_xy, tf.constant(list1)),[1,N]);  # the cast of list1 to tensor is not strictly needed
# the reshape for cr_xx is needed to have a shape compatible with matrix multiplication
print(cr_xx)

tf.Tensor([[-0.5 -1.  -1.5 -2. ]], shape=(1, 4), dtype=float32)


Return the laplacian by combinining the derivatives with Rp and Rq

In [22]:
Rq_np, Rp_np, _ = ps.RQRP(N)
RQ=tf.constant(Rq_np)
RP=tf.constant(Rp_np)
dqq = tf.matmul(cr_xx, RQ)
dpp = tf.matmul(cr_xx, RP)
lapla=dqq+dpp
print(dqq, dpp, lapla)

tf.Tensor([[-0.5 -1.5]], shape=(1, 2), dtype=float32) tf.Tensor([[-1. -2.]], shape=(1, 2), dtype=float32) tf.Tensor([[-1.5 -3.5]], shape=(1, 2), dtype=float32)


## Compare with the LaplacianLayer

In [23]:
lapla_test=ps.LaplacianLayer(N)(chir, chii, modelVacuum);  # create a laplacian layer
modelVacuumLapla = tf.keras.Model(inputs = xin, outputs=lapla_test) # add the layer to a model
print(modelVacuumLapla(xtrain)) # evaluate the model
# the output is the same as above

tf.Tensor([[-1.5 -3.5]], shape=(1, 2), dtype=float32)


# Details the operation to evaluate the Biharmonic (squared laplacian)

In [24]:
x = tf.Variable(np.zeros((1,N)),dtype=tf_real)
with tf.GradientTape() as t4:
    with tf.GradientTape() as t3:
        with tf.GradientTape() as t2:
            with tf.GradientTape() as t1:
                cr, _ = modelVacuum(x)
            cr_x = t1.gradient(cr, x)
        cr_xy = t2.jacobian(cr_x, x)
    cr_xyz = t3.jacobian(cr_xy, x)
cr_xyzw = t4.jacobian(cr_xyz, x)
#print(cr);print(cr_x);print(cr_xy);print(cr_xyz);print(cr_xyzw)

## Define the indices to extract diagonal part

In [25]:
# indeces to diagonal the biharmonic and laplacian
list_2x =[]
list_4x =[]
for i in range(N):
    list_2x.append([0,i]*2);
    list_4x.append([i]*4);
    #list_4x.append([0,i]*4);
# list_2x is [[0,0,0,0],[0,1,0,1],[0,2,0,2],[0,3,0,3]] for N=4
# list_4x is [[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3] for N=4            
indices_2x=tf.constant(list_2x) 
indices_4x=tf.constant(list_4x)
print(indices_2x, indices_4x)

tf.Tensor(
[[0 0 0 0]
 [0 1 0 1]
 [0 2 0 2]
 [0 3 0 3]], shape=(4, 4), dtype=int32) tf.Tensor(
[[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]], shape=(4, 4), dtype=int32)


## Extract the diagonal parts of laplacian and biharmonic

In [26]:
# extract the diagonal part by gather
cr_2x=tf.reshape(tf.gather_nd(cr_xy,indices_2x), [1, N])
print(cr_2x)

tf.Tensor([[-0.5 -1.  -1.5 -2. ]], shape=(1, 4), dtype=float32)


In [27]:
cr_xyzw=tf.reshape(cr_xyzw,[N,N,N,N])
cr_4x=tf.gather_nd(cr_xyzw,indices_4x)
cr_4x=tf.reshape(cr_4x,[1,N])
print(cr_4x)

tf.Tensor([[ 0.75  3.    6.75 12.  ]], shape=(1, 4), dtype=float32)


## Combine the diagonal part to have laplacian and biharmonic

In [28]:
dqq = tf.matmul(cr_2x,RQ)
dpp = tf.matmul(cr_2x,RP)
dqqqq = tf.matmul(cr_4x,RQ)
dpppp = tf.matmul(cr_4x,RP)
biharmonic = dqqqq+dpppp+2*tf.multiply(dqq,dpp)
laplacian = dqq+dpp
print(laplacian, biharmonic)

tf.Tensor([[-1.5 -3.5]], shape=(1, 2), dtype=float32) tf.Tensor([[ 4.75 24.75]], shape=(1, 2), dtype=float32)


## Compare with the Biharmonic layer

In [29]:
biharm_test=ps.BiharmonicLayer(N)(chir, chii, modelVacuum);  # create a laplacian layer
modelVacuumBiharm = tf.keras.Model(inputs = xin, outputs=biharm_test) # add the layer to a model
print(modelVacuumBiharm(xtrain)) # evaluate the model
# the output is the same as above

(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 4.75, 24.75]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-1.5, -3.5]], dtype=float32)>)


# Test the Heisenberg Layer

## Define a model with a vacuum mode and a coherent mode

In [30]:
N = 4
vacuum = ps.VacuumLayer(N) 
# this is the true vacuum, not the wrong one used above to test
# displacement vector for the coherent mode
dinput = np.zeros((N,1));
nc0 = 4 # number of photons in the coherent mode
dinput[2]=np.sqrt(nc0) 
dinput[3]=np.sqrt(nc0)
# Glauber Layer
D=ps.DisplacementLayerConstant(dinput)
# Heisenber Layer
H=ps.HeisenbergLayer(N)
# Build the model
xin = tf.keras.Input((N,))
x0, a0  = D(xin)
chir, chii = vacuum(x0,a0)
model = tf.keras.Model(inputs = xin, outputs=[chir, chii])

In [31]:
# add the Heisenberg layer
nboson, nboson2, Dn2 = H(chir, chii, model)
# create modified model including the Heisenber Layer
modelHeisenberg = tf.keras.Model(inputs = xin, outputs=[nboson, nboson2, Dn2])

## Call the HeisenberLayer

In [32]:
print(modelHeisenberg(xtrain)); 

[<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 4.]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0., 20.]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 4.]], dtype=float32)>]


Remark, the model is computationally intensive as it use the Jacobian layer

As expected we have 0 photons in the vacuum state, <br>
nc0 photons in the coherent state (4 for nc0=4) <br>
0 is the expected value for second moment of photons of the vacuum <br>
nc0+nc0^2 is second moment of photons in the coherent state (20 for nc0=4) <br>
0 is the uncertainty of photons in the vaccum <br>
nc0 is also the uncertainty of photons in the coherent state (4 for nc0=4)

# Use special Heisenberg for Gaussian states

In [33]:
HG = ps.HeisenbergGaussianLayer(N)
# add the Heisenberg Gaussian layer
nboson, nboson2, Dn2 = HG(chir, chii, model)
# create modified model including the Heisenber Layer
modelHeisenbergG = tf.keras.Model(inputs = xin, outputs=[nboson, nboson2, Dn2])

In [34]:
print(modelHeisenbergG(xtrain)); 

[<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 4.]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0., 20.]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 4.]], dtype=float32)>]


# Test the biharmonic layer with Gaussian states

In [35]:
biharm_test=ps.BiharmonicGaussianLayer(N)(chir, chii, modelVacuum);  # create a laplacian layer
modelVacuumBiharmG = tf.keras.Model(inputs = xin, outputs=biharm_test) # add the layer to a model
print(modelVacuumBiharmG(xtrain)) # evaluate the model
# the output is the same as above

(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 4.75, 24.75]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-1.5, -3.5]], dtype=float32)>)


We obtained same results as above